In [60]:
import time
import pandas as pd
from urllib.request import urlopen
import json
from bs4 import BeautifulSoup

def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)


#Biz has 10 pages
df_all = []
i=0
dataframe = pd.DataFrame(columns=['Subject', 'Comment', 'Post Number', 'Replies'])
for i in range(10):
    i=i+1
    url = (("https://a.4cdn.org/biz/") + str(i) + '.json')
    data = get_jsonparsed_data(url)

    list = len(data['threads'])
   
    for i in range(0, list): 
      try:
          comment = data['threads'][i]['posts'][0]['com']
          subject = data['threads'][i]['posts'][0]['sub']
          postno = data['threads'][i]['posts'][0]['no']
          replies = data['threads'][i]['posts'][0]['replies']
      except KeyError:
          subject = 'No subject'
          postno = data['threads'][i]['posts'][0]['no']
          replies = data['threads'][i]['posts'][0]['replies']
      except KeyError:
          comment = data['threads'][i]['posts'][0]['sub']
      dataframe = dataframe.append({'Subject':subject, 'Comment':comment, 'Post Number':postno, 'Replies':replies}, ignore_index=True)
    time.sleep(.02)
    i=i+1

dataframe

,Subject,Comment,Post Number,Replies
0,NO BEGGING,"<span style=""font-weight:600;font-size:150%;li...",4884770,0
1,Welcome to /biz/ - Business &amp; Finance,This board is for the discussion of topics rel...,21374000,1
2,No subject,"This is how you know a project is solid, WSG h...",44454512,8
3,its beginning to look a lot like christm,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA<wbr>AAAAAA...,44455298,1
4,No subject,didn’t take profits again,44454475,15
...,...,...,...,...
195,No subject,Look at the bright side. It just can&#039;t di...,44453931,1
196,who&#039;s &quot;slurping&quot; and who&#039;s...,comfy nft chad reporting a project worth looki...,44453055,1
197,No subject,"<span class=""quote"">&gt;2018</span><br>OH NO I...",44453629,1
198,No subject,What are some unpumped coins have really top n...,44454039,4


In [191]:
df2 = dataframe

for i in range(0,5):
  try:
    postno, replies = get_numbers(i) 
    url = (("https://a.4cdn.org/biz/thread/") + str(postno) + '.json')
    data = get_jsonparsed_data(url)
    replies_text = []
    print(replies_text)
    dataframe2['reply_list'][i] = url
  except HTTPError:
    dataframe2['reply_list'][i] = '404'
    print(replies_text)
    pass
 
    for j in range(replies):
      time.sleep(.5)
      try:
        reply = data['posts'][j]['com']
        replies_text.append(reply)
        print(replies_text)
      except KeyError: 
        reply = 'Key Error'
        print(replies_text)
      except IndexError:
        reply = 'Index Error'
        print(replies_text)
    print(replies_text)
    dataframe2['reply_list'][i] = replies_text


dataframe2


[]
[]
[]
[]
[]


,Subject,Comment,Post Number,Replies,reply_list
0,NO BEGGING,"<span style=""font-weight:600;font-size:150%;li...",4884770,0,https://a.4cdn.org/biz/thread/4884770.json
1,Welcome to /biz/ - Business &amp; Finance,This board is for the discussion of topics rel...,21374000,1,https://a.4cdn.org/biz/thread/21374000.json
2,Ready to end it all,I&#039;m so fucking tired bros...,44454933,7,https://a.4cdn.org/biz/thread/44454933.json
3,/XSG/ XRP Schizo General #1650,"Previously on XSG: <a href=""/biz/thread/444173...",44446784,159,https://a.4cdn.org/biz/thread/44446784.json
4,No subject,I just bought 0.1 BTC.,44452512,68,https://a.4cdn.org/biz/thread/44452512.json
...,...,...,...,...,...
195,This is just the beginning,"4 hours until the Asian market opens, you&#039...",44451760,6,1
196,No subject,"A Metaversa Mafia game, heard of it?",44452588,11,1
197,Look at it from the bright side,At least you won&#039;t have to pay any taxes ...,44453642,0,1
198,Rate my portfolio and i rate yours,"i have<br><span class=""quote"">&gt;AVAX like ev...",44452017,10,1


In [143]:
#for i in range(0,10):
url = (("https://a.4cdn.org/biz/thread/") + str(44454159) + '.json')
data = get_jsonparsed_data(url)
reply = data['posts'][5]
#print(reply)
#print(dataframe.iloc[197])
#data['threads'][18]['posts'][0]['sub']
reply
#dataframe['Post Number'][6]
#https://a.4cdn.org/biz/thread/44454159.json

{'com': '<a href="#p44454361" class="quotelink">&gt;&gt;44454361</a><br>if you&#039;re new I&#039;d say cheapest to a degree. get what you like<br>I like sunshines, krugs, monarch chunkies<br><br>US junk is the best overall to buy for SHTF, with 1oz rounds next. If you are stacking for weight, get bars. If you are stacking for divisibility, then junk/round. If you are stacking for best price, generic 1oz rounds and silverback slabs go for ~$25/ozt',
 'id': 'rA4vSy86',
 'name': 'Anonymous',
 'no': 44454499,
 'now': '12/13/21(Mon)16:46:20',
 'resto': 44454159,
 'time': 1639431980}

##some dudes working code 

'''import requests
r = requests.get('https://a.4cdn.org/pol/catalog.json')
r = r.json()
for pages in range(0, len(r)):
    for threads in r[pages]['threads']:
        print(threads)

In [39]:
dataframe2 = dataframe
dataframe2['reply_list'] = 1

In [96]:
from urllib.error import HTTPError

replies_text = []

def get_numbers(i):  
    postno = dataframe2['Post Number'][i]
    replies = dataframe2['Replies'][i]
    return(postno, replies)



#for i in range(0,len(dataframe2['Replies']))
#for i in range(len(dataframe2['Replies'])):

for i in range(0,len(dataframe2['Replies'])):
  try:
    postno, replies = get_numbers(i) 
    url = (("https://a.4cdn.org/biz/thread/") + str(postno) + '.json')
    data = get_jsonparsed_data(url)
    replies_text = []
  except HTTPError:
    dataframe2['reply_list'][i] = '404'
    pass
    for j in range(0, replies):
      try:
        reply = data['posts'][j]['com']
        replies_text.append(reply)
      except KeyError: 
        reply = 'Key Error'
      except IndexError:
        reply = 'Index Error'
    print(replies_text)
    dataframe2['reply_list'][i] = replies_text
   

#for i in range(10):
 #   i=i+1
  #  url = (("https://a.4cdn.org/biz/") + str(i) + '.json')
  #  data = get_jsonparsed_data(url)
#print(replies_text)

0      1
1      1
2      1
3      1
4      1
      ..
195    1
196    1
197    1
198    1
199    1
Name: reply_list, Length: 200, dtype: int64

In [9]:
for i in range(10):
    i=i+1
    url = (("https://a.4cdn.org/biz/") + str(i) + '.json')
    data = get_jsonparsed_data(url)
print(replies_text)

#dataframe2.tail(50)
#for j in range(0,10):
 # print('penis')
  #for i in range(0,20):
   # print(str(i))
#postno, reply = get_numbers(2)
#print(postno)
#print(reply)


[]


In [10]:
replies_text

[]

In [ ]:
#CATALOG

url = ("https://a.4cdn.org/biz/5.json")
data = get_jsonparsed_data(url)

In [ ]:

num_replies = int(data['posts'][0]['replies'])
replies = []

for i in range(num_replies):
    reply = data['posts'][i]['com']
    replies.append(reply)

replies

soup = BeautifulSoup() 
def beautify (reply): 
  soup = BeautifulSoup(reply) 
  reply = soup.get_text() 
  return reply

#df["clean_text"] = df.description.apply(lambda text: beautify(reply))

replies


['Reading charts is fucking hard enough, why do i have to suffer in the hards of this shitty exchanges as well.',
 '<a href="#p43975308" class="quotelink">&gt;&gt;43975308</a><br>Fuck you anon, you wanna trade or not?',
 '<a href="#p43975308" class="quotelink">&gt;&gt;43975308</a><br><span class="quote">&gt;Reading charts is fucking hard enough</span><br>once things get easy, the top is in.',
 '<a href="#p43975319" class="quotelink">&gt;&gt;43975319</a><br>Shut the fuck up pajeet, you obviously don&#039;t trade fag.',
 '<a href="#p43975346" class="quotelink">&gt;&gt;43975346</a><br>Perfect sell signal anon.',
 '<a href="#p43975381" class="quotelink">&gt;&gt;43975381</a><br>ID is a buy signal',
 '<a href="#p43975308" class="quotelink">&gt;&gt;43975308</a><br>Fags can&#039;t even come up with one with zero slippage.',
 '<a href="#p43975488" class="quotelink">&gt;&gt;43975488</a><br>They just keep repeating the same shitty utilities, bunch of pajeets.',
 '<a href="#p43975488" class="quote

In [ ]:
''''This is to get the replies into a long string for NLP, need to use regex too

#stringy = str(replies)
#soup = BeautifulSoup(stringy)
#print(soup.get_text())

'''



"'This is to get the replies into a long string for NLP, need to use regex too\n\n#stringy = str(replies)\n#soup = BeautifulSoup(stringy)\n#print(soup.get_text())\n\n"

In [ ]:
threads['no'][1]

In [ ]:
import requests
from datetime import datetime as dt
r = requests.get('https://a.4cdn.org/pol/catalog.json')
r = r.json()
def gen_chan():
    for idx, page in enumerate(r):
        for thread in r[idx]['threads']:
            yield thread
def get_threads(key: str, default='NaN'):
    return threads.get(key, default)
for threads in gen_chan():
    no = get_threads('no')
    now = get_threads('now')
    time = get_threads('time')
    my_time = dt.today()
    com = TextMaster.strip_html(get_threads('com'))
    name = get_threads('name')
    trip = get_threads('trip')
    ids = get_threads('id')
    capcode = get_threads('capcode')
    filename = get_threads('filename') + get_threads('ext')
    resto = get_threads('resto')
    semantic_url = get_threads('semantic_url')
    replies = get_threads('replies')
    images = get_threads('images')
    url = TextMaster.extract_url(get_threads('com'))
    sent = TextMaster.textblob_sentiment(get_threads('com'))
    if 'last_replies' in threads:
        for comment in threads['last_replies']:
            com_com = comment.get('com', 'NaN')
            resto_com = comment.get('resto', 'NaN')
            now_com = comment.get('now', 'NaN')
            time_com = comment.get('time', 'NaN')
            fname_com = comment.get('filename', 'NaN')
            url_com = comment.get('com')
            sent_com = comment.get('com')

NameError: ignored

In [ ]:
gen_chan()

<generator object gen_chan at 0x7f7302c316d0>

In [ ]:
def update_dataframe():

    for i in range(0, list): 
        try:
            comment = data['threads'][i]['posts'][0]['com']
            subject = data['threads'][i]['posts'][0]['sub']
        except KeyError:
            subject = 'No subject'
        print(subject)
        print(comment + ' ,')
        dataframe = dataframe.append({'Subject':subject, 'Comment':comment}, ignore_index=True)
    return

In [ ]:
def update_dataframe():
    dataframe = pd.DataFrame(columns=['Subject', 'Comment'])
    list = len(data['threads'])

    for i in range(0, list): 
        try:
            comment = data['threads'][i]['posts'][0]['com']
            subject = data['threads'][i]['posts'][0]['sub']
        except KeyError:
            subject = 'No subject'
        print(subject)
        print(comment + ' ,')
        dataframe = dataframe.append({'Subject':subject, 'Comment':comment}, ignore_index=True)
    return

In [ ]:
update_dataframe()

No subject
what do you faggots think about SAND? ,
No subject
how do I tell if a coin I got from an airdrop is safe to sell? i know to run it through tokensniffer but don&#039;t know what it means by the token isn&#039;t a honeypot ,
No subject
Prove to me this statement is not true: <br>The only reason ETH is mooning at the moment is because of the shitcoins on it needing a shitload of gas for fees due to how bloated ETH is. This is ETH&#039;s only utility and it is not a good one for the crypto space.<br>Prove to me I am wrong. Pro fucking tip, you can&#039;t. ,
No subject
Why is BTC just crabbing when ETH and AVAX are mooning? ,
Attentions please
Check out sloth token<br><br>0x2e6e1ccba6f2091a5a9e80e67ee18907f<wbr>f5e096c ,
No subject
<span class="quote">&gt;200 ETH bag now worth 30 eth</span><br><br>T-thanks Sergay ,
No subject
How many CRO to be her chastity slave? ,
No subject
Why is it that every coin that I buy either dumps or does absolutely nothing.<br>Regardless of fundament